In [125]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [126]:
data = {'img_path': [], 'label': []}

for root, dirs, files in os.walk("./training/"):
    label = os.path.basename(root)
    for file in files:
        if file.endswith('.jpg'):
            data['img_path'].append(os.path.join(root, file))
            data['label'].append(label)

df_data = pd.DataFrame(data)

In [166]:
def orb_sift(image, step_size=8):
    # read image from img_path then convert to gray scale
    # construct a ORB object 
    sift = cv2.ORB_create()
    # create dense keypoints and compute descriptors
    kp = [cv2.KeyPoint(x, y, step_size) for x in range(0, image.shape[0], step_size) for y in range(0, image.shape[1], step_size)]
    kp, des = sift.compute(image, kp)
    return des

In [264]:
visual_words = []

for i, row in df_data.iterrows():
    # read image from img_path then convert to gray scale
    image = cv2.imread(row['img_path'])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # get descriptors for each image
    visual_words.append(orb_sift(image, 5))
    
df_data['visual_words'] = visual_words

In [265]:
df_data.head()

,img_path,label,visual_words
0,./training/bedroom\0.jpg,bedroom,"[[0, 0, 0, 16, 0, 0, 0, 0, 16, 16, 0, 0, 0, 0,..."
1,./training/bedroom\1.jpg,bedroom,"[[35, 135, 98, 137, 214, 145, 170, 111, 15, 65..."
2,./training/bedroom\10.jpg,bedroom,"[[193, 90, 159, 126, 0, 4, 111, 182, 112, 202,..."
3,./training/bedroom\11.jpg,bedroom,"[[248, 195, 108, 186, 140, 201, 69, 239, 134, ..."
4,./training/bedroom\12.jpg,bedroom,"[[123, 170, 195, 88, 238, 177, 73, 87, 211, 83..."


In [266]:
BoVW = df_data['visual_words'].to_list()
BoVW = np.vstack(BoVW)
print('bag of visual words size:', BoVW.shape)

bag of visual words size: (1950046, 32)


In [267]:
df_data['visual_words'][1].shape

(891, 32)

In [268]:
# K-Means clustering
from sklearn.cluster import KMeans

k_means = KMeans(n_clusters = 50, random_state = 0, n_init = 1, verbose = 0)
k_means.fit(BoVW)
vocabulary = k_means.cluster_centers_

In [269]:
vocabulary.shape

(50, 32)

In [270]:
def histogram_bovw(visual_words, codebook, k):
    his = np.zeros(k)
    for vw in visual_words:
        # find distance from vw to each representation vector (codebook)
        dist = np.power(np.power(np.tile(vw, (k, 1)) - codebook, 2).sum(axis=1), 0.5)
        # min distance
        min_codebook = dist.argsort()[0]
        # calculate histogram
        his[min_codebook] += 1
    return his

In [271]:
his_bovw = []

for i, row in df_data.iterrows():
    # read image from img_path then convert to gray scale
    image = cv2.imread(row['img_path'])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    his_bovw.append(histogram_bovw(orb_sift(image, 5), vocabulary, 50))

his_bovw = np.array(his_bovw)

In [272]:
his_bovw = np.array(his_bovw)
his_bovw.shape

(1500, 50)

In [241]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, top_k_accuracy_score

In [273]:
X = his_bovw
y = np.array(df_data['label'].to_list())
print(X.shape, y.shape)

(1500, 50) (1500,)


In [289]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [316]:
import lightgbm as lgb
clf = lgb.LGBMClassifier(n_estimators = 50, learning_rate=0.2)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [317]:
print('accuracy:', accuracy_score(y_test, y_pred))

accuracy: 0.56


In [343]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X_sc = scaler.fit_transform(X)

In [365]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [366]:
clf = SVC(kernel = 'poly', degree = 3, probability = True).fit(X_train, y_train)

In [367]:
y_pred = clf.predict(X_test)
print('accuracy:', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

accuracy: 0.6133333333333333
              precision    recall  f1-score   support

       Coast       0.80      0.67      0.73        12
      Forest       0.46      0.86      0.60         7
     Highway       0.60      0.75      0.67         8
  Insidecity       0.82      0.64      0.72        14
    Mountain       0.50      0.38      0.43        13
      Office       0.56      0.45      0.50        11
 OpenCountry       0.73      0.67      0.70        12
      Street       0.70      0.78      0.74         9
      Suburb       0.60      0.75      0.67        12
TallBuilding       0.89      0.89      0.89         9
     bedroom       0.50      0.50      0.50         8
  industrial       0.40      0.25      0.31         8
     kitchen       0.57      0.36      0.44        11
  livingroom       0.44      0.50      0.47         8
       store       0.54      0.88      0.67         8

    accuracy                           0.61       150
   macro avg       0.61      0.62      0.60       1

In [368]:
top_k_accuracy_score(y_test, clf.predict_proba(X_test), k=2)

0.8

In [369]:
confusion_matrix(y_test, y_pred)

array([[8, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 6, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 6, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 9, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 5, 1, 0, 5, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 5, 0, 0, 2, 0, 0, 2, 1, 1, 0],
       [0, 0, 0, 1, 2, 0, 8, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 9, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 8, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 4, 0, 2, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 2],
       [0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 1, 0, 4, 2, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 4, 2],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 7]], dtype=int64)

In [144]:
from sklearn.model_selection import GridSearchCV

In [407]:
# parameters = {'kernel':['poly'], 'degree': [2,3],'C':np.linspace(0.1, 3, 50), 'gamma': [0.1, 1.0, 10, 100]}
#parameters = {'kernel':['rbf'], 'C':np.linspace(0.001, 10, 50), 'gamma': [0.1, 1.0, 10, 100]}
# parameters = {'C': [1, 5, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']}

In [408]:
# grid_search = GridSearchCV(SVC(), parameters)
# grid_search.fit(X_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': array([0.1       , 0.15918367, 0.21836735, 0.27755102, 0.33673469,
       0.39591837, 0.45510204, 0.51428571, 0.57346939, 0.63265306,
       0.69183673, 0.75102041, 0.81020408, 0.86938776, 0.92857143,
       0.9877551 , 1.04693878, 1.10612245, 1.16530612, 1.2244898 ,
       1.28367347, 1.34285714, 1.40204082, 1.46122449, 1.52040816,
       1.57959184, 1.63877551, 1.69795918, 1.75714286, 1.81632653,
       1.8755102 , 1.93469388, 1.99387755, 2.05306122, 2.1122449 ,
       2.17142857, 2.23061224, 2.28979592, 2.34897959, 2.40816327,
       2.46734694, 2.52653061, 2.58571429, 2.64489796, 2.70408163,
       2.76326531, 2.82244898, 2.88163265, 2.94081633, 3.        ]),
                         'degree': [2, 3], 'gamma': [0.1, 1.0, 10, 100],
                         'kernel': ['poly']})

In [410]:
# grid_search.best_score_

0.5281481481481481

In [406]:
# grid_search.best_params_

{'C': 2.0530612244897957, 'degree': 2, 'kernel': 'rbf'}

In [149]:
# y_pred = grid_search.predict(X_test)
# print('accuracy:', accuracy_score(y_test, y_pred))
# print(classification_report(y_test, y_pred))

accuracy: 0.08666666666666667
              precision    recall  f1-score   support

       Coast       0.00      0.00      0.00        13
      Forest       0.00      0.00      0.00        15
     Highway       0.00      0.00      0.00        23
  Insidecity       0.00      0.00      0.00        19
    Mountain       0.09      1.00      0.16        26
      Office       0.00      0.00      0.00        16
 OpenCountry       0.00      0.00      0.00        29
      Street       0.00      0.00      0.00        20
      Suburb       0.00      0.00      0.00        17
TallBuilding       0.00      0.00      0.00        19
     bedroom       0.00      0.00      0.00        22
  industrial       0.00      0.00      0.00        17
     kitchen       0.00      0.00      0.00        24
  livingroom       0.00      0.00      0.00        18
       store       0.00      0.00      0.00        22

    accuracy                           0.09       300
   macro avg       0.01      0.07      0.01       

C:\Users\pooja\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pooja\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pooja\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [150]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB

In [374]:
clf = GaussianNB().fit(X_train, y_train)

In [375]:
y_pred = clf.predict(X_test)
print('accuracy:', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

accuracy: 0.47333333333333333
              precision    recall  f1-score   support

       Coast       0.54      0.58      0.56        12
      Forest       0.50      0.71      0.59         7
     Highway       0.62      0.62      0.62         8
  Insidecity       0.75      0.43      0.55        14
    Mountain       0.57      0.31      0.40        13
      Office       0.25      0.09      0.13        11
 OpenCountry       0.40      0.50      0.44        12
      Street       0.47      0.78      0.58         9
      Suburb       0.47      0.75      0.58        12
TallBuilding       0.78      0.78      0.78         9
     bedroom       0.40      0.50      0.44         8
  industrial       0.00      0.00      0.00         8
     kitchen       0.33      0.09      0.14        11
  livingroom       0.38      0.38      0.38         8
       store       0.33      0.75      0.46         8

    accuracy                           0.47       150
   macro avg       0.45      0.48      0.44       

In [411]:
top_k_accuracy_score(y_test, clf.predict_proba(X_test), k=2)

0.66

In [419]:
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier

rbf_feature = RBFSampler(random_state=1)
X_features = rbf_feature.fit_transform(X)
clf = SGDClassifier(max_iter=200)
clf.fit(X_features, y)
clf.score(X_features, y)

0.18333333333333332

In [420]:
from sklearn.svm import LinearSVC

clf2 = LinearSVC().fit(X_train, y_train)
y_pred = clf2.predict(X_test)
print('accuracy:', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

accuracy: 0.24666666666666667
              precision    recall  f1-score   support

       Coast       0.50      0.33      0.40        12
      Forest       0.33      0.57      0.42         7
     Highway       0.44      0.50      0.47         8
  Insidecity       1.00      0.14      0.25        14
    Mountain       0.60      0.23      0.33        13
      Office       0.00      0.00      0.00        11
 OpenCountry       1.00      0.08      0.15        12
      Street       0.50      0.11      0.18         9
      Suburb       0.38      0.25      0.30        12
TallBuilding       1.00      0.22      0.36         9
     bedroom       0.12      0.75      0.20         8
  industrial       0.00      0.00      0.00         8
     kitchen       0.00      0.00      0.00        11
  livingroom       0.00      0.00      0.00         8
       store       0.15      0.88      0.25         8

    accuracy                           0.25       150
   macro avg       0.40      0.27      0.22       

C:\Users\pooja\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\pooja\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pooja\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pooja\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being se

In [424]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
clf2 = make_pipeline(StandardScaler(), LinearSVC(random_state=0,  tol=1e-4))
clf2.fit(X_train, y_train)
y_pred = clf2.predict(X_test)
print('accuracy:', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

accuracy: 0.5466666666666666
              precision    recall  f1-score   support

       Coast       0.70      0.58      0.64        12
      Forest       0.46      0.86      0.60         7
     Highway       0.78      0.88      0.82         8
  Insidecity       0.90      0.64      0.75        14
    Mountain       0.62      0.38      0.48        13
      Office       0.67      0.36      0.47        11
 OpenCountry       0.60      0.50      0.55        12
      Street       0.50      0.67      0.57         9
      Suburb       0.59      0.83      0.69        12
TallBuilding       0.89      0.89      0.89         9
     bedroom       0.40      0.50      0.44         8
  industrial       0.18      0.25      0.21         8
     kitchen       0.38      0.27      0.32        11
  livingroom       0.00      0.00      0.00         8
       store       0.50      0.62      0.56         8

    accuracy                           0.55       150
   macro avg       0.54      0.55      0.53       1

C:\Users\pooja\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
